## Análise de Comunidades Baseada no Foursquare

### Disciplina: análise de redes sociais

### Equipe:
* Henrique Lima
* Julio Sales
* Mácio Matheus
* Victor Outtes

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import requests
import json
import itertools
from pandas.io.json import json_normalize
import community

## 1. Coleta de dados

A coleta foi feita utilizando a API fornecida pelo Foursquare. Ela necessita de 2 parâmetros para ser acessada, o CLIENT ID e o CLIENT SECRETE, que podem ser obtidos ao se cadastrar como desenvolvedor no site da ferramenta.

O acesso que utilizamos utiliza coordenadas do local desejado e um raio de busca para encontrar os estabelecimentos mais curtidos. Esta foi a base da consulta.

Após isto, fazemos mais uma consulta: coletar os usuários que deram curtidas em todos os estabelecimentos obtidos. Juntando as 2 bases, conseguimos gerar um CSV. A partir dele que tudo será feito.

#### Função auxiliar para obter a categoria do local

In [2]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    return categories_list[0]['name']

#### Dados básicos utilizados nas consultas

In [ ]:
CLIENT_ID = '55LWIFZLZNZPSTAZZQKJXBD11BSBCUZTZPIXWVV4RFZJP1HS' # your Foursquare ID
CLIENT_SECRET = 'RPCOWGHQY5YNQUCHQUTC5VLKEYFGZN15FSIYTK5GNXNZ301E' # your Foursquare Secret
VERSION = '20180604'
latitude_nyork = 40.785091
longitude_nyork = -73.968285
radius = 10000
LIMIT = 100

#### Obtem a lista de locais

In [ ]:
QUERY = ''
url = 'https://api.foursquare.com/v2/venues/explore?time=any&query={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    QUERY, CLIENT_ID, CLIENT_SECRET, latitude_nyork, longitude_nyork, VERSION, radius, LIMIT)
results = requests.get(url).json()
locais = results['response']['groups'][0]['items']
locais_proximos = json_normalize(locais)
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
locais_proximos = locais_proximos.loc[:, filtered_columns]
locais_proximos.columns = [col.split(".")[-1] for col in locais_proximos.columns]
locais_proximos['categories'] = locais_proximos.apply(get_category_type, axis=1)
locais_proximos.tail(10)

#### Para cada local pega as curtidas

In [ ]:
checkins_locais = pd.DataFrame()
for i,local in locais_proximos.iterrows():
    url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}&limit={}'.format(
    local['id'], CLIENT_ID, CLIENT_SECRET, VERSION, 10000)
    results = requests.get(url).json()
    if (results['response']['likes']['count'] > 0) & ('items' in results['response']['likes']):
        results_2 = results['response']['likes']['items']
        checkins = json_normalize(results_2)[['id', 'firstName', 'gender']]
        checkins['avenue_id'] = local['id']
        checkins['avenue_name'] = local['name']
        checkins['avenue_categories'] = local['categories']
        checkins['avenue_lat'] = local['lat']
        checkins['avenue_lng'] = local['lng']
        if checkins_locais.empty:
            checkins_locais = checkins
        else:
            checkins_locais = pd.concat([checkins_locais, checkins], ignore_index=True)

In [ ]:
# checkins_locais.to_csv('likes_NY_geral.csv', index=False)
checkins_locais = pd.read_csv('likes_NY_geral.csv')
checkins_locais.tail()

## 2. Pré-processamento dos dados

#### Filtragens

Aqui pegamos os locais com até 650 curtidas, para evitar pegar locais mais famosos que todo mundo vai. O ideal é utilizar a base toda, mas por questões de infraestrutura para processamento decidimos este caminho.

In [ ]:
locais_vc = checkins_locais['avenue_id'].value_counts()
locais_filter = locais_vc[locais_vc <= 650].index.tolist()
len(locais_filter)

Retirando da base de dados apenas os locais identificados acima...

In [ ]:
likes_completo = checkins_locais[checkins_locais['avenue_id'].isin(locais_filter)]

Aqui filtramos da base anterior só os usuários que deram pelo menos 2 curtidas em algum lugar. Isto serve para reduzir bastante a quantidade de usuários que serão os nós do grafo.

Depois disto, pegamos a lista de locais distintos obtidos depois desses 2 filtros.

In [ ]:
vc = likes_completo['id'].value_counts()
filter = vc[vc >= 2].index.tolist()
locais_unicos = likes_completo[likes_completo['id'].isin(filter)]['avenue_id'].drop_duplicates()
locais_unicos = locais_unicos.tolist()
len(locais_unicos)

## 3. Criação do grafo

Aqui segue a criação do grafo. Primeiro criamos os nós com os atributos de cada usuário. Após, para cada local selecionado, pegamos a listagem de usuários que deram curtida e fazemos uma combinação 2 a 2 para criar as arestas. 

Se a aresta entre dois nós não existe ela é criada com peso 1. Se já existe, o peso é incrementado em 1.

In [ ]:
grafo = nx.Graph()
for usuarios in usuarios_unicos:
    grafo.add_node(str(usuarios), 
           nome=likes_completo[likes_completo['id']==usuarios]['firstName'].values[0],
           genero=likes_completo[likes_completo['id']==usuarios]['gender'].values[0])
for local in locais_unicos:
    # pessoas que deram like...
    pessoas = list(likes_completo[
        (likes_completo['avenue_id'] == local) & (likes_completo['id'].isin(filter))
    ]['id'].drop_duplicates().values)
    combinacoes = itertools.combinations(pessoas, 2)
    for comb in combinacoes:
        u = comb[0]
        outro = comb[1]
        if grafo.has_edge(str(u), str(outro)):
            grafo[str(u)][str(outro)]['weight'] = grafo[str(u)][str(outro)]['weight'] + 1
        else:
            grafo.add_edge(str(u), str(outro), weight=1)

In [ ]:
# nx.write_gml(grafo, 'grafo_new_york.gml')
grafo = nx.read_gml('grafo_new_york.gml')

## 4. Análise básica

Número de nós

In [ ]:
nx.number_of_nodes(grafo)

Número de arestas

In [ ]:
nx.number_of_edges(grafo)

Grau médio

In [ ]:
np.mean([grau[1] for grau in list(nx.degree(grafo, weight='weight'))])

Centralidade de grau média

In [ ]:
np.mean(list(nx.degree_centrality(grafo).values()))

Centralidade de autovetor média

In [ ]:
np.mean(list(nx.eigenvector_centrality(grafo, weight='weight').values()))

Centralidade de proximidade média

In [ ]:
np.mean(list(nx.betweenness_centrality(grafo, weight='weight').values()))

Coeficiente de cluster médio

In [ ]:
nx.average_clustering(grafo, weight='weight')

Excentricidade média

In [ ]:
np.mean(list(nx.eccentricity(grafo).values()))

## 5. Visualização

Calcula o posicionamento dos nós

In [ ]:
pos = nx.spring_layout(grafo)

Desenha o grafo. As cores são formadas a partir das medidas de centralidade de grau de cada nó. Os tamanhos são definidos proporcionalmente às medidas de centralidade de autovetor de cada nó.

In [ ]:
node_color = [cent for cent in nx.degree_centrality(grafo).values()]
node_size = [eigen * 1000 for eigen in list(nx.eigenvector_centrality(grafo, weight='weight').values())]
plt.figure(figsize=(15,15))
ed = nx.draw_networkx_edges(grafo, pos=pos, alpha=0.3, edge_color='gray')
no = nx.draw_networkx_nodes(grafo, pos=pos, alpha=0.6, with_labels=False, node_color=node_color, 
                            cmap=cm.jet, vmin=min(list(nx.degree_centrality(grafo).values())), 
                            vmax=max(list(nx.degree_centrality(grafo).values())),
                            node_size=node_size)
plt.colorbar(no)
plt.axis('off')

## 6. Detecção de comunidades

Para detectar as eventuais comunidades da rede utilizamos a biblioteca <strong>python-louvain</strong>. Ele define as comunidades com apenas a linha de código a seguir:

In [ ]:
partition = community.best_partition(grafo, weight='weight')

Agora vamos desenhar o grafo de forma bem parecida com o anterior. Porém, agora, as cores serão as comunidades armazenadas na variável <strong>partition</strong>

In [ ]:
node_color = [partition.get(node) for node in grafo.nodes()]
node_size = [eigen * 1000 for eigen in list(nx.eigenvector_centrality(grafo, weight='weight').values())]
plt.figure(figsize=(15,15))
ed = nx.draw_networkx_edges(grafo, pos=pos, alpha=0.3, edge_color='gray')
no = nx.draw_networkx_nodes(grafo, pos=pos, alpha=0.6, with_labels=False, node_color=node_color, 
                            cmap=cm.jet, vmin=0.0, vmax=max(partition.values()),
                            node_size=node_size)
plt.colorbar(no)
plt.axis('off')

## 6.1. Análise das comunidades

A partir da definição dos grupos, podemos atualizar nossa base de dados inicial de curtidas com o grupo de cada usuário:

In [ ]:
likes_completo['cluster'] = likes_completo['id'].apply(
    lambda x: partition[str(x)] if str(x) in list(partition.keys()) else None)
likes_com_cluster = likes_completo[~likes_completo['cluster'].isnull()]
likes_com_cluster.tail()

In [ ]:
#likes_com_cluster.to_csv('likes_NY_geral_cluster.csv', index=False)

Para cada comunidade detectada podemos ver quais foram os locais mais curtidos pelos usuários:

In [ ]:
likes_com_cluster[likes_com_cluster['cluster'] == 0]['avenue_name'].value_counts()[:3]

In [ ]:
likes_com_cluster[likes_com_cluster['cluster'] == 1]['avenue_name'].value_counts()[:3]

In [ ]:
likes_com_cluster[likes_com_cluster['cluster'] == 2]['avenue_name'].value_counts()[:3]

In [ ]:
likes_com_cluster[likes_com_cluster['cluster'] == 3]['avenue_name'].value_counts()[:3]

In [ ]:
likes_com_cluster[likes_com_cluster['cluster'] == 4]['avenue_name'].value_counts()[:3]

In [ ]:
likes_com_cluster[likes_com_cluster['cluster'] == 5]['avenue_name'].value_counts()[:3]

Também podemos encontrar as pessoas mais influentes da rede, pegando os nós com maior centralidade:

In [ ]:
influentes = []
dic = nx.degree_centrality(grafo)
for v in sorted(dic, key=dic.get, reverse=True):
    influentes.append(v)
likes_com_cluster[likes_com_cluster['id'].isin(influentes[:10])][['firstName', 'gender', 'cluster']].drop_duplicates()